In [18]:
import os
from google.cloud import aiplatform
from google.cloud import storage
import pandas as pd
from kfp import compiler, dsl

# --- Variables de Entorno y Configuración ---
PROJECT_ID = 'projectstylus01'
REGION = 'us-central1'
SERVICE_ACCOUNT = 'workbench-sa@projectstylus01.iam.gserviceaccount.com'
# Su bucket de GCS donde se almacenarán los artifacts y datos procesados.
GCS_BUCKET = 'gs://mit-project-vertex-ai-artifacts'
PIPELINE_ROOT = f'{GCS_BUCKET}/pipeline_root/'
PIPELINE_FILE = 'fraud_model_pipeline_experiments.yaml'

# Inicializar Vertex AI
aiplatform.init(
    project=PROJECT_ID, 
    location=REGION, 
    service_account=SERVICE_ACCOUNT, 
    staging_bucket=GCS_BUCKET
)

In [19]:
# Definier el nombre del experimento
experiment = 'E1'

In [20]:
import pandas as pd
import numpy as np

# Cargar los datos
data = pd.read_csv('gs://mit-proyect-data/Base.csv')

# Definiendo variables nan y categoricas
var_nan = ['prev_address_months_count','current_address_months_count','intended_balcon_amount',
           'bank_months_count','session_length_in_minutes','device_distinct_emails_8w']

# Ingenieria de variables
data[var_nan] = data[var_nan].replace(-1, np.nan).astype('float')

data['prev_address_valid'] = np.where(data['prev_address_months_count'] > 0,1,0)
data['velocity_6h'] = np.where(data['velocity_6h'] <= 0,data["velocity_6h"].quantile(0.25),data["velocity_6h"])
data['ratio_velocity_6h_24h'] = data['velocity_6h']/data['velocity_24h']
data['ratio_velocity_24h_4w'] = data['velocity_24h']/data['velocity_4w']
data['log_bank_branch_count_8w'] = np.log1p(data['bank_branch_count_8w'])
data['log_days_since_request'] = np.log1p(data['days_since_request'])
data['prev_bank_months_count'] = np.where(data['bank_months_count'] <=0, 0, 1)
data['income_risk_score'] = data['income']*data['credit_risk_score']

data = data.drop(columns = ['device_fraud_count','month','prev_address_months_count','intended_balcon_amount', 'source'])

# Guardar los datos procesados

DATA_PROCESS_URI = f'{GCS_BUCKET}/data_process/data-{experiment}.csv'
data.to_csv(DATA_PROCESS_URI, index=False)
print(f"Datos procesados guardados en: {DATA_PROCESS_URI}")

Datos procesados guardados en: gs://mit-project-vertex-ai-artifacts/data_process/data-E1.csv


In [21]:
# Ejecutar si el cofig-template.yaml se modifico para el experimento
import yaml
from google.cloud import storage

def config_parameters(experiment_name: str):
    
    template = 'config-template.yaml'
    
    with open(template, 'r') as f:
        config = yaml.safe_load(f)

    display(config)

    client = storage.Client()
    bucket = client.bucket('mit-project-vertex-ai-artifacts')
    destination_blob_name = f'config/experiments/config-{experiment_name}.yaml'
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(template)
    print(f"Configuracion de hiperparametros subido a gs://mit-project-vertex-ai-artifacts/{destination_blob_name} listo para usar en el pipeline")


config_parameters(experiment)

{'RandomForestClassifier': {'n_estimators': [100, 500],
  'max_depth': [5, 20],
  'max_features': ['auto', 'sqrt', 'log2'],
  'min_samples_split': [2, 10],
  'min_samples_leaf': [1, 5],
  'bootstrap': [True, False],
  'class_weight': ['None', 'balanced', 'balanced_subsample']},
 'LGBMClassifier': {'objective': 'binary',
  'max_depth': [3, 10],
  'learning_rate': [0.01, 0.3],
  'feature_fraction': [0.5, 1.0],
  'bagging_fraction': [0.5, 1.0],
  'num_leaves': [20, 100],
  'lambda_l1': [0.0, 1.0],
  'lambda_l2': [0.0, 1.0]},
 'XGBClassifier': {'objective': 'binary:logistic',
  'max_depth': [3, 6],
  'learning_rate': [0.01, 0.3],
  'n_estimators': [50, 300],
  'subsample': [0.5, 1.0],
  'colsample_bytree': [0.5, 1.0],
  'reg_alpha': [0.0, 1.0],
  'reg_lambda': [0.0, 1.0],
  'gamma': [0.0, 1.0],
  'eval_metric': 'aucpr'}}

Configuracion de hiperparametros subido a gs://mit-project-vertex-ai-artifacts/config/experiments/config-E1.yaml listo para usar en el pipeline


In [28]:
# Se ejecuta el Job.
import time
from google.cloud import aiplatform

# Definición de Parámetros del Pipeline
pipeline_parameters = {
    # IMPORTANTE: Se inyecta el URI de la data procesada en el paso anterior
    'raw_data_path': DATA_PROCESS_URI, 
    'params_config_path': f'gs://mit-project-vertex-ai-artifacts/config/experiments/config-{experiment}.yaml',
    'model_display_name': 'fraud-model-experiment',
    'train_size': 0.8,
    'val_size': 0.1,
    'test_size': 0.1,
    'n_trials': 5, # Usar un valor más bajo para pruebas rápidas
    'human_hit_rate': 0.85, # Tasa de acierto del revisor humano
}

# Crear y Ejecutar el Pipeline Job
job = aiplatform.PipelineJob(
    display_name='custom-training-run',
    template_path=PIPELINE_FILE,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=pipeline_parameters,
    enable_caching=False, # Recomendado para pruebas con data nueva
)

print("Iniciando Vertex AI Pipeline Job...")
job.run(sync=False) # Ejecución asíncrona (no bloquea el notebook)

time.sleep(10)
print(f"\n➡️ Revise el estado del Job aquí: {job.resource_name}")

Iniciando Vertex AI Pipeline Job...
Creating PipelineJob
PipelineJob created. Resource name: projects/435304534790/locations/us-central1/pipelineJobs/mit-project-banking-pipeline-training-20251006072120
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/435304534790/locations/us-central1/pipelineJobs/mit-project-banking-pipeline-training-20251006072120')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mit-project-banking-pipeline-training-20251006072120?project=435304534790
PipelineJob projects/435304534790/locations/us-central1/pipelineJobs/mit-project-banking-pipeline-training-20251006072120 current state:
PipelineState.PIPELINE_STATE_RUNNING

➡️ Revise el estado del Job aquí: projects/435304534790/locations/us-central1/pipelineJobs/mit-project-banking-pipeline-training-20251006072120
PipelineJob projects/435304534790/locations/us-central1/pipelineJobs/mit-project-banking-pipeline-training